# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Data-Pre-Processing" data-toc-modified-id="Data-Pre-Processing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data Pre-Processing</a></div><div class="lev2 toc-item"><a href="#Load-Data" data-toc-modified-id="Load-Data-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load Data</a></div><div class="lev2 toc-item"><a href="#Word-Segmentation" data-toc-modified-id="Word-Segmentation-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Word Segmentation</a></div>

#  Data Pre-Processing

## Load Data

In [1]:
import os

In [2]:
def load_data(path):
    """
    Load date from file
    """
    input_file = os.path.join(path)
    with open(input_file, 'r', encoding='utf-8') as f:
        return f.read()

In [17]:
train_data_path = 'train_data.txt'
train_data = load_data(train_data_path).strip().split('\t')[1:]
train_data = [line.split('\n')[:2] for line in train_data]
test_data_path = 'test_data.txt'
test_data = load_data(test_data_path).strip().split('\t')[1:]
test_data = [line.split('\n')[:2] for line in test_data]

In [24]:
train_sent = [line[0] for line in train_data]
test_sent = [line[0] for line in test_data]
train_label = ['Causal' if line[-1] == 'Cause-Effect(e2,e1)' or line[-1] == 'Cause-Effect(e1,e2)' else 'Non-Causal' for line in train_data]
test_label = ['Causal' if line[-1] == 'Cause-Effect(e2,e1)' or line[-1] == 'Cause-Effect(e1,e2)' else 'Non-Causal' for line in test_data]

In [32]:
#len([i for i in train_label if i == 'Causal']) # 1003
#len([i for i in test_label if i == 'Causal']) # 328

## Word Segmentation

In [59]:
import re
from nltk import regexp_tokenize

In [60]:
def clean1(raw_list):
    """
    Merge eneity
    """
    for i in range(len(raw_list)):
        if len(raw_list[i]) == 3:
            for j in range(3):
                if not re.search('<e1>.+</e1>', raw_list[i][j]) and not re.search('<e2>.+</e2>', raw_list[i][j]):
                    raw_list[i][j] = raw_list[i][j] + '+' + raw_list[i][j+1]
                    del raw_list[i][j+1]
                    break
        if len(raw_list[i]) == 4:
             raw_list[i][0] = raw_list[i][0] + '+' + raw_list[i][1]
             raw_list[i][1] = raw_list[i][2] + '+' + raw_list[i][3]
             raw_list[i].pop()
             raw_list[i].pop()
    return raw_list

def clean2(raw_list):
    """
    Remove "'s"'<e1>''</e1>''<e2>''</e2>'
    """
    for i in range(len(raw_list)):
        for j in range(2):
            if raw_list[i][j].endswith('s'):
                raw_list[i][j] = raw_list[i][j][:-2]
    new_list = [[w[4:-5] for w in s] for s in raw_list]
    return new_list

def substitute(sentences, eneities):
    """
    Substitute the raw eneities of sentences with eneities
    """
    for i in range(len(sentences)):
        sentences[i], _ = re.subn('<e1>.+\s?.+</e1>', eneities[i][0], sentences[i])
        sentences[i], _ = re.subn('<e2>.+\s?.+</e2>', eneities[i][1], sentences[i])
    return sentences     

def cut(s):
    """
    Word segmentation
    """
    pattern = r'''
              (?x)                   # set flag to allow verbose regexps 
              (?:[A-Z]\.)+           # abbreviations, e.g. U.S.A. 
              |\d+(?:\.\d+)?%?       # numbers, incl. currency and percentages 
              |\w+(?:[-&']\w+)*      # words w/ optional internal hyphens/apostrophe 
           '''  
    return regexp_tokenize(s, pattern=pattern)

In [61]:
trainWords = [cut(s) for s in train_sent] 

In [62]:
trainWords[0]

['The',
 'system',
 'as',
 'described',
 'above',
 'has',
 'its',
 'greatest',
 'application',
 'in',
 'an',
 'arrayed',
 'e1',
 'configuration',
 'e1',
 'of',
 'antenna',
 'e2',
 'elements',
 'e2']

In [63]:
raw_eneities = [w for w in trainWords[0] 
                 if re.search('<e1>', w) or re.search('</e1>', w) or re.search('<e2>', w) or re.search('</e2>', w)] 

In [64]:
raw_eneities

[]